In [ ]:
# save as scrape_token2049_partners.py
import re
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd

BASE = "https://www.asia.token2049.com"
HEADERS = {"User-Agent": "MyCompanyBot/1.0 (+https://yourcompany.example)"}


def fetch(url: str) -> str:
    """Fetch HTML content from a URL with error handling."""
    print(f"🌐 Fetching URL: {url}")
    r = requests.get(url, headers=HEADERS, timeout=15)
    r.raise_for_status()
    print(f"✅ Fetched {len(r.text)} characters from page")
    return r.text


def parse_partners(html: str):
    """Extract company/partner names and clean out non-company entries, with debug prints."""
    soup = BeautifulSoup(html, "html.parser")
    exhibitors = []

    blacklist = [
        "token2049", "conference", "panel", "stage", "logo", "handle",
        "audience", "media", "app", "blue", "purple", "mascot"
    ]

    for idx, el in enumerate(soup.select(".sponsor, .partner, .logo, .sponsors, img[alt]"), start=1):
        # Get name
        name = (el.get("alt") or el.get_text(strip=True)).strip()

        if not name:
            print(f"{idx}: ⚠️ Empty name, skipped")
            continue

        lower_name = name.lower()
        if any(term in lower_name for term in blacklist):
            print(f"{idx}: ⚠️ Blacklist skipped: {name}")
            continue

        if len(name) < 2 or name.isdigit():
            print(f"{idx}: ⚠️ Too short or numeric, skipped: {name}")
            continue

        exhibitors.append({"name": name})
        print(f"{idx}: ✅ Added: {name}")

    # Deduplicate
    seen = set()
    dedup = []
    for e in exhibitors:
        name = e["name"]
        if name.lower() in seen:
            print(f"⚠️ Duplicate skipped: {name}")
            continue
        seen.add(name.lower())
        dedup.append(e)

    print(f"📌 Total unique companies found: {len(dedup)}")
    return dedup



def main():
    print("🔍 Fetching TOKEN2049 partners page...")
    url = BASE + "/partners"
    html = fetch(url)

    print("📊 Parsing company names and visitors info...")
    rows = parse_partners(html)

    # Save results
    df = pd.DataFrame(rows)
    df.to_excel("token2049_exhibitors.xlsx", index=False)

    print(f"✅ Done! Saved {len(rows)} rows to token2049_exhibitors.xlsx")


if __name__ == "__main__":
    main()

🔍 Fetching TOKEN2049 partners page...
🌐 Fetching URL: https://www.asia.token2049.com/partners
✅ Fetched 219832 characters from page
📊 Parsing company names and visitors info...
1: ⚠️ Blacklist skipped: Token2049 conference panel on stage in Singapore with blue lighting
2: ⚠️ Empty name, skipped
3: ⚠️ Blacklist skipped: Donald Trump JR at TOKEN2049
4: ⚠️ Blacklist skipped: Arthur Hayes at TOKEN2049
5: ⚠️ Blacklist skipped: Large Audience at TOKEN2049
6: ⚠️ Empty name, skipped
7: ⚠️ Empty name, skipped
8: ✅ Added: OKX
9: ✅ Added: BloFin
10: ✅ Added: Coinbase
11: ✅ Added: Bingx
12: ✅ Added: TRON
13: ✅ Added: Mesh Connect
14: ✅ Added: Bitget
15: ✅ Added: DWF Labs
16: ✅ Added: MetaEarth
17: ✅ Added: Spacecoin
18: ✅ Added: Bitstamp by Robinhood
19: ✅ Added: Solana
20: ✅ Added: Sui
21: ✅ Added: CoinW
22: ✅ Added: MEXC
23: ✅ Added: Auros
24: ✅ Added: EVG
25: ✅ Added: KuCoin
26: ✅ Added: Gemini
27: ✅ Added: TON
28: ✅ Added: Bit Digital
29: ✅ Added: Stacks
30: ✅ Added: Input Output
31: ✅ Added: 

In [ ]:
# save as scrape_token2049_partners_enriched.py
import re
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urlparse

BASE = "https://www.asia.token2049.com"
HEADERS = {"User-Agent": "MyCompanyBot/1.0 (+https://yourcompany.example)"}

def fetch(url: str) -> str:
    print(f"\n🌐 [FETCH] URL: {url}")
    try:
        r = requests.get(url, headers=HEADERS, timeout=15)
        print(f"📶 Status Code: {r.status_code}")
        r.raise_for_status()
        print(f"✅ [FETCHED] {len(r.text)} characters from {url}")
        return r.text
    except Exception as e:
        print(f"❌ [ERROR] Failed to fetch {url}: {e}")
        return ""

def parse_partners(html: str):
    print("🔍 [PARSING] Start parsing partner elements...")
    soup = BeautifulSoup(html, "html.parser")
    exhibitors = []

    blacklist = [
        "token2049", "conference", "panel", "stage", "logo", "handle",
        "audience", "media", "app", "blue", "purple", "mascot"
    ]

    elements = soup.select(".sponsor, .partner, .logo, .sponsors, img[alt]")
    print(f"🔹 Total elements found: {len(elements)}")

    for idx, el in enumerate(elements, start=1):
        name = (el.get("alt") or el.get_text(strip=True)).strip()
        print(f"{idx:03} Element text/alt: '{name}'")

        if not name:
            print(f"⚠️ Empty name skipped")
            continue

        lower_name = name.lower()
        if any(term in lower_name for term in blacklist):
            print(f"❌ Blacklisted: {name}")
            continue

        if len(name) < 2 or name.isdigit():
            print(f"⚠️ Too short/numeric: {name}")
            continue

        print(f"✅ Added: {name}")
        exhibitors.append({"name": name})

    # Deduplicate
    seen = set()
    dedup = []
    for e in exhibitors:
        name = e["name"]
        if name.lower() in seen:
            print(f"⚠️ Duplicate skipped: {name}")
            continue
        seen.add(name.lower())
        dedup.append(e)

    print(f"📌 Total unique companies found after deduplication: {len(dedup)}")
    return dedup

def search_serper(company_name: str) -> str:
    """Search for the official site using Serper.dev (Google search API)."""
    query = f"{company_name} official site"
    print(f"\n🔎 [Serper] Searching: {query}")

    from dotenv import load_dotenv
    import os

    load_dotenv()  
    api_key = os.getenv("api_key")

    url = "https://google.serper.dev/search"
    headers = {"X-API-KEY": api_key, "Content-Type": "application/json"}
    payload = {"q": query}

    try:
        r = requests.post(url, json=payload, headers=headers, timeout=15)
        r.raise_for_status()
        results = r.json().get("organic", [])
        print(f"🔹 Serper returned {len(results)} results")
        for result in results:
            link = result.get("link", "")
            if link and not any(s in link for s in ["linkedin.com", "twitter.com", "facebook.com"]):
                print(f"🔗 [Serper] Found website: {link}")
                return link
    except Exception as e:
        print(f"❌ Serper search failed for {company_name}: {e}")

    return ""




def clean_email(email: str) -> str:
    """Return a valid, usable email or empty string if invalid/suspicious."""
    if not email:
        return ""

    email = email.strip()
    email_lower = email.lower()

    # Basic email pattern
    if not re.match(r"^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$", email_lower):
        return ""

    # ✅ Expanded placeholder, generic, or test/probe emails
    placeholders = {
        "user@domain.com", "info@example.com", "email@domain.com",
        "contact@example.com", "admin@example.com", "someone@example.com",
        "name@domain.com", "name@gmail.com", "example@domain.com",
        "j.appleseed@example.com", "email@provider.com", "email@helixar.com",
        "email@email.com", "you@example.com", "jane.diaz@example.com",
        "filler@godaddy.com", "email@site.com", "asxvmprobertest@gmail.com",
        "smart.journey.prober@gmail.com", "your@email.com", "john@example.com",
        "you@company.com", "partner@agency.com", "jane@email.com",
        "example@mysite.com", "you@email.com"
    }
    if email_lower in placeholders:
        return ""

    # ✅ System-generated or weird unicode-prefixed emails
    system_prefixes = [
        "noreply@", "no-reply@", "donotreply@", "do-not-reply@",
        "auto@", "notifications@", "u003e", "u2028", "u002f@"
    ]
    if any(email_lower.startswith(p) for p in system_prefixes):
        return ""

    # ✅ Suspicious or malformed domain patterns
    invalid_keywords = [
        "telegram", "instagram", "insagram", "applicability"
    ]
    domain_part = email_lower.split("@")[-1]
    if any(kw in domain_part for kw in invalid_keywords):
        return ""

    # ✅ Non-standard TLD (too long, e.g., >10 chars)
    tld = domain_part.split(".")[-1]
    if len(tld) > 10:
        return ""

    # ✅ File-like or suspicious extensions
    suspicious_exts = [
        ".png", ".jpg", ".jpeg", ".svg", ".gif", ".webp",
        ".mjs", ".patch", ".min.js"
    ]
    if any(email_lower.endswith(ext) for ext in suspicious_exts):
        return ""

    # ✅ Autogenerated hash-like emails
    if re.match(r"^[0-9a-f]{10,}@", email_lower):
        return ""

    # ✅ Emails with 8+ consecutive digits
    if re.search(r"[0-9]{8,}", email_lower):
        return ""

    # ✅ Remove invisible / unicode chars
    email_cleaned = re.sub(r"[\u2000-\u206F\u2E00-\u2E7F\u2028\u2029]", "", email_lower)

    return email_cleaned


def extract_emails_from_website(base_url: str, max_emails: int = 5) -> list[str]:
    """Extract clean, usable emails from homepage + common subpages."""
    if not base_url:
        print("⚠️ No URL provided")
        return []

    urls_to_check = [base_url]
    parsed = urlparse(base_url)
    domain = parsed.netloc
    scheme = parsed.scheme or "https"
    subpages = ["contact", "about", "support", "privacy", "legal", "terms"]
    for slug in subpages:
        urls_to_check.append(f"{scheme}://{domain}/{slug}")

    found_emails = []

    for url in urls_to_check:
        try:
            print(f"\n🌐 Fetching URL: {url}")
            r = requests.get(url, headers=HEADERS, timeout=(15, 20))
            r.raise_for_status()
            emails = re.findall(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}", r.text)
            print(f"🔹 Emails found on page: {emails}")
            for e in emails:
                valid_email = clean_email(e)
                if valid_email and valid_email not in found_emails:
                    found_emails.append(valid_email)
                    print(f"✅ Added email: {valid_email}")
                    if len(found_emails) >= max_emails:
                        return found_emails
        except Exception as ex:
            print(f"⚠️ Could not fetch {url}: {ex}")
            continue

    if not found_emails:
        print("⚠️ No valid email found on any page")
    return found_emails



def main():
    print("🚀 [START] Scraping TOKEN2049 partners...\n")
    html = fetch(BASE + "/partners")
    if not html:
        print("❌ Failed to fetch main page. Exiting.")
        return

    print("\n📊 [STEP 1] Extracting partner names...")
    rows = parse_partners(html)
    if not rows:
        print("⚠️ No partners found. Exiting.")
        return

    #rows = rows[298:350]

    print("\n🔍 [STEP 2] Enriching with website + email...\n")
    enriched = []

    for i, row in enumerate(rows, start=1):
        name = row["name"]
        print(f"\n🔹 [{i}/{len(rows)}] Processing: {name}")

        website = search_serper(name)
        print(f"🌐 Website: {website if website else 'Not found'}")

        if not website:
            print(f"⚠️ No website found for {name}")
            emails = []
        else:
            emails = extract_emails_from_website(website, max_emails=3)
            print(f"📬 Emails: {emails if emails else 'Not found'}")
            enriched.append({
                "name": name,
                "website": website,
                "email": ", ".join(emails) if emails else ""
            })

        # ✅ Correct indentation — runs after each company is processed
        time.sleep(2)


    df = pd.DataFrame(enriched)
    df.to_excel("token2049_enriched_companies.xlsx", index=False)
    print("\n✅ [DONE] Results saved to: token2049_enriched_companies.xlsx")

if __name__ == "__main__":
    main()


Streaming output truncated to the last 5000 lines.
🔹 Emails found on page: []

🌐 Fetching URL: https://www.digitalshield.net/about
🔹 Emails found on page: ['jchurch@digitalshield.net']
✅ Added email: jchurch@digitalshield.net

🌐 Fetching URL: https://www.digitalshield.net/support
⚠️ Could not fetch https://www.digitalshield.net/support: 404 Client Error: Not Found for url: https://www.digitalshield.net/support

🌐 Fetching URL: https://www.digitalshield.net/privacy
⚠️ Could not fetch https://www.digitalshield.net/privacy: 404 Client Error: Not Found for url: https://www.digitalshield.net/privacy

🌐 Fetching URL: https://www.digitalshield.net/legal
⚠️ Could not fetch https://www.digitalshield.net/legal: 404 Client Error: Not Found for url: https://www.digitalshield.net/legal

🌐 Fetching URL: https://www.digitalshield.net/terms
⚠️ Could not fetch https://www.digitalshield.net/terms: 404 Client Error: Not Found for url: https://www.digitalshield.net/terms
📬 Emails: ['jchurch@digitalshield.